In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc,confusion_matrix
import matplotlib.pyplot as plt

# データの読み込み
train_data_sig = pd.read_csv('selected_train_set.csv')
test_data_sig = pd.read_csv('selected_test_set.csv')

# 特徴量（X）と目標変数（y）の設定
X_train_sig = train_data_sig.drop(columns=['PassengerId','Survived'])
y_train_sig = train_data_sig['Survived']
X_test_sig = test_data_sig.drop(columns=['PassengerId','Survived'])
y_test_sig = test_data_sig['Survived']

# データの標準化
scaler_sig = StandardScaler()
X_train_sig_scaler = scaler_sig.fit_transform(X_train_sig)
X_test_sig_scaler = scaler_sig.transform(X_test_sig)


#C:0.001,0.01,0.1,1,10,100,gamma:0.0001,0.001,0.01,0.1,1で0.5
param_grid_sig = {
    'C': [0.001, 0.01, 0.1, 1, 10,100],
    'gamma': [0.0001, 0.001, 0.01, 0.1,1]
}
# RBF→sigカーネルのSVMでハイパーパラメータチューニングclass_weight='balanced'を加えて、不均衡データに対応→0.5から0.56になった！！！！！
svm_sig = SVC(kernel='sigmoid', probability=True, random_state=42,class_weight='balanced')#これが一番！！！
#SVC(kernel='poly', probability=True, random_state=42,class_weight='balanced')0.43エラーも出た
#SVC(kernel='rbf', probability=True, random_state=42,class_weight='balanced')0.56 (最初に試した)
grid_search_sig = GridSearchCV(svm_sig, param_grid_sig, cv=10, scoring='accuracy')
grid_search_sig.fit(X_train_sig_scaler, y_train_sig)

# 最適なハイパーパラメータでモデルを再訓練
best_sig_model = grid_search_sig.best_estimator_
best_sig_model.fit(X_train_sig_scaler, y_train_sig)

# RBFカーネルモデルの評価
y_pred_sig = best_sig_model.predict(X_test_sig_scaler)
accuracy_sig = accuracy_score(y_test_sig, y_pred_sig)

# テストデータでの予測確率を取得
# `predict_proba`を使用してクラス1（positiveクラス）の確率を取得
y_scores_sig = best_sig_model.predict_proba(X_test_sig_scaler)[:, 1]

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_sig= np.sort(np.unique(y_scores_sig))

# 感度（TPR）と特異度（1 - FPR）を計算
tpr_list_sig = []
specificity_list_sig = []

for threshold in thresholds_sig:
    # スコアをしきい値で2値化
    y_pred_sig = (y_scores_sig >= threshold).astype(int)
    
    # 混同行列を取得
    tn_sig, fp_sig, fn_sig, tp_sig = confusion_matrix(y_test_sig, y_pred_sig).ravel()
    
    # 感度と特異度を計算
    tpr_sig = tp_sig / (tp_sig+ fn_sig) if (tp_sig + fn_sig) > 0 else 0  # 感度
    specificity_sig = tn_sig / (tn_sig + fp_sig) if (tn_sig + fp_sig) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_sig.append(tpr_sig)
    specificity_list_sig.append(specificity_sig)

# AUCの計算
roc_auc_sig = auc(np.array(specificity_list_sig), tpr_list_sig)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_sig), tpr_list_sig, label=f"ROC Curve (AUC = {roc_auc_sig:.2f})", color="blue")

plt.plot([1, 0], [0, 1], color="gray", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()


# RBFカーネルSVMの結果表示
print("最適化されたsigmoidカーネルSVMの精度:", accuracy_sig)
print("sigmoidカーネルSVM最適パラメータ:", grid_search_sig.best_params_)
print("sigmoidカーネルSVMの詳細な評価結果:\n", classification_report(y_test_sig, y_pred_sig))

In [ ]:
from sklearn.metrics import roc_curve, auc,confusion_matrix
import matplotlib.pyplot as plt

# テストデータでの予測確率を取得
# `predict_proba`を使用してクラス1（positiveクラス）の確率を取得
y_scores_sig = best_sig_model.predict_proba(X_test_sig_scaler)[:, 1]

# ROC曲線の計算（FPRとTPRを取得）
fpr_sig, tpr_sig, thresholds_sig = roc_curve(y_test_sig, y_scores_sig)
specificity_sig = 1 - fpr_sig  # 特異度の計算
roc_auc_sig = auc(fpr_sig, tpr_sig)  # AUCの計算

# ROC曲線の描画（特異度 vs 感度）
plt.figure(figsize=(8, 6))
plt.plot(specificity_sig, tpr_sig, color="orange", label=f"ROC Curve (AUC = {roc_auc_sig:.2f})")
plt.plot([1, 0], [0, 1], color="gray", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.xlabel("Specificity (1 - FPR)")
plt.ylabel("Sensitivity (TPR)")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 混同行列の計算
cm = confusion_matrix(y_test_sig, y_pred_sig)

# 混同行列の表示
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 混同行列の要素を使って感度と特異度を計算
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)  # 感度
specificity = tn / (tn + fp)  # 特異度

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")